<a href="https://www.kaggle.com/code/syedasimalishah/running-pandas-on-gpu?scriptVersionId=108807080" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-oct-2022/sample_submission.csv
/kaggle/input/tabular-playground-series-oct-2022/train_9.csv
/kaggle/input/tabular-playground-series-oct-2022/train_3.csv
/kaggle/input/tabular-playground-series-oct-2022/train_4.csv
/kaggle/input/tabular-playground-series-oct-2022/test_dtypes.csv
/kaggle/input/tabular-playground-series-oct-2022/train_8.csv
/kaggle/input/tabular-playground-series-oct-2022/train_dtypes.csv
/kaggle/input/tabular-playground-series-oct-2022/train_5.csv
/kaggle/input/tabular-playground-series-oct-2022/train_2.csv
/kaggle/input/tabular-playground-series-oct-2022/test.csv
/kaggle/input/tabular-playground-series-oct-2022/train_0.csv
/kaggle/input/tabular-playground-series-oct-2022/train_7.csv
/kaggle/input/tabular-playground-series-oct-2022/train_1.csv
/kaggle/input/tabular-playground-series-oct-2022/train_6.csv


In [2]:
! nvidia-smi

Sat Oct 22 05:02:02 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla T4            Off  | 00000000:00:05.0 Off |                    0 |
| N/A   

In [3]:
! git clone https://github.com/rapidsai/rapidsai-csp-utils.git
! python rapidsai-csp-utils/colab/env-check.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 300, done.
remote: Counting objects: 100% (129/129), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 300 (delta 74), reused 99 (delta 55), pack-reused 171
Receiving objects: 100% (300/300), 87.58 KiB | 2.58 MiB/s, done.
Resolving deltas: 100% (136/136), done.
***********************************************************************
Woo! Your instance has the right kind of GPU, a Tesla T4!
***********************************************************************



In [ ]:
! bash rapidsai-csp-utils/colab/update_gcc.sh
import os
os._exit(00)

Updating your Colab environment.  This will restart your kernel.  Don't Panic!
Get:1 http://packages.cloud.google.com/apt gcsfuse-focal InRelease [5389 B]
Get:2 https://packages.cloud.google.com/apt cloud-sdk InRelease [6751 B]       
Get:3 https://packages.cloud.google.com/apt google-fast-socket InRelease [5405 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1581 B]
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]      
Hit:6 http://archive.ubuntu.com/ubuntu focal InRelease                         
Get:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]        
Get:8 http://ppa.launchpad.net/ubuntu-toolchain-r/test/ubuntu focal InRelease [17.5 kB]
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [740 kB]
Get:11 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packag

In [4]:
!python rapidsai-csp-utils/colab/install_rapids.py stable

Found existing installation: cffi 1.15.0
Uninstalling cffi-1.15.0:
  Successfully uninstalled cffi-1.15.0
Found existing installation: cryptography 37.0.2
Uninstalling cryptography-37.0.2:
  Successfully uninstalled cryptography-37.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.1/427.1 kB 3.9 MB/s eta 0:00:00
Installing RAPIDS Stable 21.12
Starting the RAPIDS install on Colab.  This will take about 15 minutes.

DirectoryNotACondaEnvironmentError: The target directory exists, but it is not a conda environment.
Use 'conda create' to convert the directory to a conda environment.
  target directory: /usr/local


RAPIDS conda installation complete.  Updating Colab's libraries...


In [2]:
import cudf

In [3]:
cc = ['ball_pos_x', 'ball_pos_y', 'ball_pos_z', 'ball_vel_x',
       'ball_vel_y', 'ball_vel_z', 'p0_pos_x', 'p0_pos_y', 'p0_pos_z',
       'p0_vel_x', 'p0_vel_y', 'p0_vel_z', 'p0_boost', 'p1_pos_x', 'p1_pos_y',
       'p1_pos_z', 'p1_vel_x', 'p1_vel_y', 'p1_vel_z', 'p1_boost', 'p2_pos_x',
       'p2_pos_y', 'p2_pos_z', 'p2_vel_x', 'p2_vel_y', 'p2_vel_z', 'p2_boost',
       'p3_pos_x', 'p3_pos_y', 'p3_pos_z', 'p3_vel_x', 'p3_vel_y', 'p3_vel_z',
       'p3_boost', 'p4_pos_x', 'p4_pos_y', 'p4_pos_z', 'p4_vel_x', 'p4_vel_y',
       'p4_vel_z', 'p4_boost', 'p5_pos_x', 'p5_pos_y', 'p5_pos_z', 'p5_vel_x',
       'p5_vel_y', 'p5_vel_z', 'p5_boost', 'boost0_timer', 'boost1_timer',
       'boost2_timer', 'boost3_timer', 'boost4_timer', 'boost5_timer','team_A_scoring_within_10sec','team_B_scoring_within_10sec']

In [4]:
dtypes_df = pd.read_csv('/kaggle/input/tabular-playground-series-oct-2022/train_dtypes.csv')
dtypes = {k: v for (k, v) in zip(dtypes_df.column, dtypes_df.dtype)}
df = cudf.read_csv('/kaggle/input/tabular-playground-series-oct-2022/train_0.csv',usecols=cc)
df1 = cudf.read_csv('/kaggle/input/tabular-playground-series-oct-2022/train_1.csv',usecols=cc)
df2 = cudf.read_csv('/kaggle/input/tabular-playground-series-oct-2022/train_2.csv',usecols=cc)
df3 = cudf.read_csv('/kaggle/input/tabular-playground-series-oct-2022/train_3.csv',usecols=cc)
df4 = cudf.read_csv('/kaggle/input/tabular-playground-series-oct-2022/train_4.csv',usecols=cc)
#df5 = cudf.read_csv('/kaggle/input/tabular-playground-series-oct-2022/train_5.csv',usecols=cc)
#df6 = cudf.read_csv('/kaggle/input/tabular-playground-series-oct-2022/train_6.csv',usecols=cc)
#df7 = cudf.read_csv('/kaggle/input/tabular-playground-series-oct-2022/train_7.csv',usecols=cc)
#df8 = cudf.read_csv('/kaggle/input/tabular-playground-series-oct-2022/train_8.csv',usecols=cc)
#df9 = cudf.read_csv('/kaggle/input/tabular-playground-series-oct-2022/train_9.csv',usecols=cc)

In [5]:
df.dropna(inplace=True)
df1.dropna(inplace=True)
df2.dropna(inplace=True)
df3.dropna(inplace=True)
df4.dropna(inplace=True)
#df5.dropna(inplace=True)
#df6.dropna(inplace=True)
#df7.dropna(inplace=True)
#df8.dropna(inplace=True)
#df9.dropna(inplace=True)

In [6]:
f1 = cudf.concat([df,df1,df2,df3,df4])

In [7]:
del df,df1,df2,df3,df4

In [8]:
#f2 = cudf.concat([df5,df6,df7,df8,df9])

In [9]:
#del df5,df6,df7,df8,df9

In [10]:
dft = cudf.read_csv('/kaggle/input/tabular-playground-series-oct-2022/test.csv')

In [11]:
#df = cudf.concat([f1,f2])
df=f1
del f1

In [12]:
trainXA = df[['ball_pos_x', 'ball_pos_y', 'ball_pos_z', 'ball_vel_x',
       'ball_vel_y', 'ball_vel_z', 'p0_pos_x', 'p0_pos_y', 'p0_pos_z',
       'p0_vel_x', 'p0_vel_y', 'p0_vel_z', 'p0_boost', 'p1_pos_x', 'p1_pos_y',
       'p1_pos_z', 'p1_vel_x', 'p1_vel_y', 'p1_vel_z', 'p1_boost', 'p2_pos_x',
       'p2_pos_y', 'p2_pos_z', 'p2_vel_x', 'p2_vel_y', 'p2_vel_z', 'p2_boost',
       'p3_pos_x', 'p3_pos_y', 'p3_pos_z', 'p3_vel_x', 'p3_vel_y', 'p3_vel_z',
       'p3_boost', 'p4_pos_x', 'p4_pos_y', 'p4_pos_z', 'p4_vel_x', 'p4_vel_y',
       'p4_vel_z', 'p4_boost', 'p5_pos_x', 'p5_pos_y', 'p5_pos_z', 'p5_vel_x',
       'p5_vel_y', 'p5_vel_z', 'p5_boost', 'boost0_timer', 'boost1_timer',
       'boost2_timer', 'boost3_timer', 'boost4_timer', 'boost5_timer']]
trainYA = df[['team_A_scoring_within_10sec']]

In [13]:
trainXB = df[['ball_pos_x', 'ball_pos_y', 'ball_pos_z', 'ball_vel_x',
       'ball_vel_y', 'ball_vel_z', 'p0_pos_x', 'p0_pos_y', 'p0_pos_z',
       'p0_vel_x', 'p0_vel_y', 'p0_vel_z', 'p0_boost', 'p1_pos_x', 'p1_pos_y',
       'p1_pos_z', 'p1_vel_x', 'p1_vel_y', 'p1_vel_z', 'p1_boost', 'p2_pos_x',
       'p2_pos_y', 'p2_pos_z', 'p2_vel_x', 'p2_vel_y', 'p2_vel_z', 'p2_boost',
       'p3_pos_x', 'p3_pos_y', 'p3_pos_z', 'p3_vel_x', 'p3_vel_y', 'p3_vel_z',
       'p3_boost', 'p4_pos_x', 'p4_pos_y', 'p4_pos_z', 'p4_vel_x', 'p4_vel_y',
       'p4_vel_z', 'p4_boost', 'p5_pos_x', 'p5_pos_y', 'p5_pos_z', 'p5_vel_x',
       'p5_vel_y', 'p5_vel_z', 'p5_boost', 'boost0_timer', 'boost1_timer',
       'boost2_timer', 'boost3_timer', 'boost4_timer', 'boost5_timer']]

trainYB = df[['team_B_scoring_within_10sec']]

In [14]:
test = dft[['ball_pos_x', 'ball_pos_y', 'ball_pos_z', 'ball_vel_x',
       'ball_vel_y', 'ball_vel_z', 'p0_pos_x', 'p0_pos_y', 'p0_pos_z',
       'p0_vel_x', 'p0_vel_y', 'p0_vel_z', 'p0_boost', 'p1_pos_x', 'p1_pos_y',
       'p1_pos_z', 'p1_vel_x', 'p1_vel_y', 'p1_vel_z', 'p1_boost', 'p2_pos_x',
       'p2_pos_y', 'p2_pos_z', 'p2_vel_x', 'p2_vel_y', 'p2_vel_z', 'p2_boost',
       'p3_pos_x', 'p3_pos_y', 'p3_pos_z', 'p3_vel_x', 'p3_vel_y', 'p3_vel_z',
       'p3_boost', 'p4_pos_x', 'p4_pos_y', 'p4_pos_z', 'p4_vel_x', 'p4_vel_y',
       'p4_vel_z', 'p4_boost', 'p5_pos_x', 'p5_pos_y', 'p5_pos_z', 'p5_vel_x',
       'p5_vel_y', 'p5_vel_z', 'p5_boost', 'boost0_timer', 'boost1_timer',
       'boost2_timer', 'boost3_timer', 'boost4_timer', 'boost5_timer']]

In [15]:
test.fillna(test.mean(), inplace=True)

In [16]:
del df,dft

In [17]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
f23 = sc.fit_transform(trainXA.as_gpu_matrix())
trainXA = pd.DataFrame(f23)

In [18]:
f24 = sc.fit_transform(test.as_gpu_matrix())
test = pd.DataFrame(f24)

In [19]:
del f23,f24

In [18]:
trainXA.head()

,0,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,50,51,52,53
0,0.003329,-0.003837,-1.325506,0.000312,-0.000948,-0.007836,0.921488,-0.556850,-0.483708,0.268154,...,-0.389788,-0.339445,-0.005897,-0.688623,0.821955,0.829904,0.959369,0.957177,0.833136,0.823919
1,0.003329,-0.003837,-1.325506,0.000312,-0.000948,-0.007836,0.931535,-0.565860,-0.483708,0.251581,...,-0.537479,-0.420486,-0.005834,-0.445659,0.821955,0.829904,0.959369,0.957177,0.833136,0.823919
2,0.003329,-0.003837,-1.325506,0.000312,-0.000948,-0.007836,0.952768,-0.593303,-0.359945,0.270562,...,-0.775832,-0.459969,0.977035,-0.555110,0.821955,0.829904,0.959369,0.957177,0.833136,0.823919
3,0.003329,-0.003837,-1.325506,0.000312,-0.000948,-0.007836,0.967535,-0.613363,-0.291341,0.504233,...,-0.692522,-1.040473,0.774841,-0.658106,0.821955,0.829904,0.959369,0.957177,0.833136,0.823919
4,0.003329,-0.003837,-1.325506,0.000312,-0.000948,-0.007836,0.990897,-0.645248,-0.217918,0.505994,...,-0.786240,-1.070061,0.203732,-0.761982,0.821955,0.829904,0.959369,0.957177,0.833136,0.823919


In [21]:
from xgboost import XGBClassifier
modelA = XGBClassifier(tree_method='gpu_hist', gpu_id=0)
modelA.fit(trainXA,trainYA)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=0, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [22]:
modelB = XGBClassifier(tree_method='gpu_hist', gpu_id=0)
modelB.fit(trainXB,trainYB)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=0, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [36]:
A = modelA.predict(test)
B = modelB.predict(test)

In [37]:
ss = pd.read_csv('../input/tabular-playground-series-oct-2022/sample_submission.csv')

In [38]:
df = pd.DataFrame({'id':ss.id,'team_A_scoring_within_10sec':A,'team_B_scoring_within_10sec':B})

In [40]:
df.to_csv('cudaf1.csv',index = False)

In [41]:
modelA.save_model('modelA.json')
modelB.save_model('modelB.json')

In [20]:
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
model = Sequential()
model.add(Dense(512, kernel_initializer='normal',activation='relu',input_shape=(54,)))
model.add(Dense(512,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(1,activation = 'sigmoid'))

model.summary()
model.compile(loss=BinaryCrossentropy, optimizer=Adam(lr=0.0001), metrics=['accuracy'])

2022-10-22 08:03:34.784116: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               28160     
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_4 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_5 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 1

2022-10-22 08:03:34.785031: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-22 08:03:34.791054: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-22 08:03:34.791695: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-22 08:03:34.792412: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-22 08:03:34.793098: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

In [21]:
trainYA=trainYA.to_pandas()

In [ ]:
model.fit(trainXA,trainYA,epochs=350,batch_size=55555)